## This file will navigate to a directory with multiple CSVs and compile ALL of them into 1 master CSV, with the header

In [19]:
# import dependencies
import pandas as pd
import os
import glob

# path = 'Data/Geocoded/'
path_files = glob.glob(os.path.join("*.csv"))
df = pd.concat((pd.read_csv(f) for f in path_files)).drop_duplicates().reset_index(drop=True)

# clean up dataframe
df = df.drop(columns=[
    'Acres',
    'Parcel  Class ',
    'Tax District',
    'Appraised  Value ',
    'Qualified Sales'
])

# rename columns
new_column_names = {
    'Year  Built ': 'yr_blt',
    'Sale Date': 'date',
    'HeatedSquare Ft': 'sf',
    'Sale Price':'price'
}

df.rename(columns=new_column_names, inplace=True)

# drop rows that aren't considered to be arm's lenght sales for 1 reason or another
df = df[df['Reason']!='MULTI PARCEL']
df = df[df['Reason']!='CONTIGUOUS PROPERTY']
df = df[df['Reason']!='IMPROVED AFTER SALE']
df = df[df['Reason']!='COURTHOUSE STEPS']
df = df[df['Reason']!='QUITCLAIM']

# create numeric sales column
df['price'] = df['price'].str.replace('[$,]', '', regex=True).astype(float)

# remove rows with a '0' value for sf
df = df[df['sf'] != 0]

# create necessary additional columns
df['full_address'] = df['Address'] + ' Henry County, GA'
df['unique_ID'] = df['Parcel ID'] + '-' + df['date'] + '-' + df['price'].astype(str)
df['price_sf'] = df['price'] / df['sf']

# compare the yr_blt value to when the property sold. Want to weed out all sales with a home constructed after the year of sale
df['date'] = pd.to_datetime(df['date'])
df['sale_year'] = df['date'].dt.year
df = df[df['yr_blt'] < df['sale_year']]

# create the columns that will be used by the chart
df['year'] = pd.DatetimeIndex(df['date']).year
df['month'] = pd.DatetimeIndex(df['date']).month
df['year-month'] = df['year'].astype(str) + '-' + df['month'].astype(str)

# drop any ridiculously small structures
df = df[df['sf'] >=500]

# finally, drop the 'Reason' column
df = df.drop(columns='Reason')

# df.to_csv('../Geocode/Henry_ready4geocode.csv')

# check to see if there are any lot sales, that is, built AFTER year it sold?
df[(df['year'] == 2020) & (df['yr_blt'] == 2019)]

# df

,Parcel ID,Address,date,price,yr_blt,sf,full_address,unique_ID,price_sf,sale_year,year,month,year-month
2472,094D02294000,752 GALVESTON WAY,2020-09-25,208300.0,2019,1831,"752 GALVESTON WAY Henry County, GA",094D02294000-9/25/2020-208300.0,113.762971,2020,2020,9,2020-9
2544,074D01419000,2804 BIRMINGHAM DR,2020-09-18,234000.0,2019,2008,"2804 BIRMINGHAM DR Henry County, GA",074D01419000-9/18/2020-234000.0,116.533865,2020,2020,9,2020-9
2593,053F01037000,249 JANNEY CIR,2020-09-30,295900.0,2019,2871,"249 JANNEY CIR Henry County, GA",053F01037000-9/30/2020-295900.0,103.065134,2020,2020,9,2020-9
2602,028G01085000,1524 GALLUP DR,2020-09-30,283900.0,2019,2237,"1524 GALLUP DR Henry County, GA",028G01085000-9/30/2020-283900.0,126.911042,2020,2020,9,2020-9
2620,146F01149000,721 PETALUMA PL,2020-09-29,327900.0,2019,2433,"721 PETALUMA PL Henry County, GA",146F01149000-9/29/2020-327900.0,134.771887,2020,2020,9,2020-9
...,...,...,...,...,...,...,...,...,...,...,...,...,...
29486,110C01246000,92 SHENANDOAH DR,2020-06-05,327800.0,2019,3089,"92 SHENANDOAH DR Henry County, GA",110C01246000-6/5/2020-327800.0,106.118485,2020,2020,6,2020-6
29487,138B01017000,153 DELWOOD DR,2020-06-05,397500.0,2019,3695,"153 DELWOOD DR Henry County, GA",138B01017000-6/5/2020-397500.0,107.577808,2020,2020,6,2020-6
29510,138B01099000,525 NOBLEWOOD DR,2020-06-04,344000.0,2019,3694,"525 NOBLEWOOD DR Henry County, GA",138B01099000-6/4/2020-344000.0,93.123985,2020,2020,6,2020-6
29548,185C01009000,133 CLEAR SPRINGS DR,2020-06-01,299900.0,2019,3260,"133 CLEAR SPRINGS DR Henry County, GA",185C01009000-6/1/2020-299900.0,91.993865,2020,2020,6,2020-6
